In [8]:
import requests

response = requests.get('http://127.0.0.1:5000/hello')
print(response.text)

data = {'a': 1, 'b': 2, 'c' : 3}
response = requests.post('http://127.0.0.1:5000/api', json=data)
print(response.json())

Hello, World!
{'a': 1, 'b': 4, 'c': 9}


In [1]:
import pandas as pd
import requests
from io import StringIO

# URL for the CSV file
url = 'https://reg.bom.gov.au/climate/dwo/202404/text/IDCJDW3050.202404.csv'

# Send HTTP request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Skip the initial 7 lines of metadata, then read the content into a pandas DataFrame
    data = pd.read_csv(StringIO(response.text), skiprows=7)
    print(data.head())  # Print the first few rows of the DataFrame to verify it's reading correctly
else:
    print(f"Failed to retrieve data: {response.status_code}")


   Unnamed: 0       Date  Minimum temperature (°C)  Maximum temperature (°C)  \
0         NaN  2024-04-1                      14.0                      28.2   
1         NaN  2024-04-2                      14.6                      21.4   
2         NaN  2024-04-3                      12.5                      16.6   
3         NaN  2024-04-4                       9.5                      18.9   
4         NaN  2024-04-5                      11.5                      20.6   

   Rainfall (mm)  Evaporation (mm)  Sunshine (hours)  \
0            0.0               3.6               2.3   
1           53.2               7.0               6.2   
2            2.2               3.0               1.0   
3            5.2               1.8               6.4   
4            0.0               5.0               7.5   

  Direction of maximum wind gust   Speed of maximum wind gust (km/h)  \
0                               N                               43.0   
1                              SW     